# Programming Assignment 4: Data Science Practicum

Name: Harrison Jacob

Date: 4/1/2025

Summary: This notebook contains code that will allow the user to gather different data regarding alcohol licensing in the Fayette county area of Kentucky. Each chunk of code and output should be described properly within the notebook

---



This first section imports all required libraries and allows the user to designate their desired file name for input

In [106]:
#Imports required
import pandas as pd
import requests
import time

# Properly read and alter the files that will be used in the code
# You are free to change the files as you see fit. I created a new .csv file with a shortened amount of data for time sake that I will include in the submission
file_nq = "ActiveLicensesNQ-Shortened-2024 - ActiveLicensesNQ-2024.csv"
file_all = "ActiveLicenses-2024.csv"
# You may need to change how many rows are skipped depending on your file
df_nq = pd.read_csv(file_nq, skiprows=3)
df_all = pd.read_csv(file_all, skiprows=3)

print("Data Imported!")

Data Imported!


This second section creates the population dataframe that will be used to hold all of the statistics from the cencus API.

In [107]:
#Function to create the population dataframe
def get_tract_population():
    # Grab the url
    pop_url = 'https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP05_0001E&for=tract:*&in=state:21&in=county:067'
    # Handle the data
    response = requests.get(pop_url)
    raw_pop_data = response.json()
    # Put the data into a dataframe
    pop_data = raw_pop_data[1:]
    pop_df = pd.DataFrame(pop_data, columns=raw_pop_data[0])
    # Grab only the name of the cencus tract
    pop_df['Tract_ID'] = pop_df['NAME'].str.extract('([^,]+)')
    pop_df.rename(columns={'DP05_0001E': 'Population'}, inplace=True)
    print("Population dataframe data input successful!")
    return pop_df[['Tract_ID', 'Population']]

# Call the function and properly format it
population_df = get_tract_population()
population_df.dropna(inplace=True)
population_df['Population'] = population_df['Population'].astype(int)
# print(population_df)

Population dataframe data input successful!


This third section allows us to convert all of the street addresses into their responding tract

In [111]:
# Function to get Census Tract using the US Census Geocoding API
def get_census_tract(street):
    # Call the global variable
    global count
    # URL for API
    base_url = f'https://geocoding.geo.census.gov/geocoder/geographies/address?street={street}&city=Lexington&state=KY&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=6&format=json'
    try:
        # Grab the data
        response = requests.get(base_url)
        data = response.json()
        # Format the data
        tract_data = data['result']['addressMatches'][0]['geographies']['Census Tracts'][0]
        tract_name = tract_data['NAME']
        tract_id = tract_data['GEOID']

        # you can add time.sleep(1) if API is overloaded

    # Error Handling
    except (requests.exceptions.RequestException, IndexError, KeyError):
        print(f"Error processing address: {street}")
        return None
    # Print successsful addition
    print("{} addresses succesfully converted!".format(count))
    count += 1
    return tract_name

count = 0
df_nq['Tract_ID'] = df_nq['PremisesStreet'].apply(get_census_tract)

0 addresses succesfully converted!
1 addresses succesfully converted!
2 addresses succesfully converted!
3 addresses succesfully converted!
4 addresses succesfully converted!
5 addresses succesfully converted!
6 addresses succesfully converted!
7 addresses succesfully converted!
8 addresses succesfully converted!
9 addresses succesfully converted!
10 addresses succesfully converted!
11 addresses succesfully converted!
Error processing address: 122 Marion Ste 120
12 addresses succesfully converted!
13 addresses succesfully converted!
14 addresses succesfully converted!
15 addresses succesfully converted!
16 addresses succesfully converted!
17 addresses succesfully converted!
18 addresses succesfully converted!
19 addresses succesfully converted!


This final section merges and visualizes the data that the user would like to see

In [112]:
# Merge and compute rates
tract_counts = df_nq['Tract_ID'].value_counts().reset_index()
tract_counts.columns = ['Tract_ID', 'License_Count']
df_merged = tract_counts.merge(population_df, on='Tract_ID', how='left')
df_merged.dropna(inplace=True)
df_merged['Rate_per_1000'] = (df_merged['License_Count'] / df_merged['Population']) * 1000

# Top 20 by rate
top_20_rate = df_merged.nlargest(20, 'Rate_per_1000')
# Top 20 by count
top_20_count = df_merged.nlargest(20, 'License_Count')

# Display results
print("Top 20 Census Tracts by Alcohol License Rate:")
print(top_20_rate[['Tract_ID', 'Rate_per_1000']])
print("\nTop 20 Census Tracts by Number of Licenses:")
print(top_20_count[['Tract_ID', 'License_Count']])


Top 20 Census Tracts by Alcohol License Rate:
              Tract_ID  Rate_per_1000
0    Census Tract 1.01       0.946074
9      Census Tract 10       0.625782
8      Census Tract 22       0.587199
6      Census Tract 13       0.469263
11      Census Tract 3       0.404531
3       Census Tract 7       0.383142
2       Census Tract 9       0.336078
1   Census Tract 32.02       0.306185
12  Census Tract 42.07       0.283206
13  Census Tract 39.11       0.280505
7       Census Tract 2       0.270563
10     Census Tract 24       0.228258
14  Census Tract 34.02       0.221680
4       Census Tract 6       0.218866
5   Census Tract 42.04       0.179308
15  Census Tract 42.08       0.130361

Top 20 Census Tracts by Number of Licenses:
              Tract_ID  License_Count
0    Census Tract 1.01              3
1   Census Tract 32.02              2
2       Census Tract 9              2
3       Census Tract 7              1
4       Census Tract 6              1
5   Census Tract 42.04             